# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Configure Logger

In [7]:
from scrabu import generate_shipment_numbers 
from scrabu import request
from scrabu import html_to_json
from scrabu import shipment_details
from scrabu import save_dictionary
from scrabu import process_shipment_number
import concurrent

In [8]:
# Multi-threading
def main(shipment_number=None, size=None, max_workers=None, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    
    shipment_numbers = generate_shipment_numbers(shipment_number=shipment_number, size=size)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(process_shipment_number, sn): sn for sn in shipment_numbers}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (url, exc))

In [9]:
list_of_shipment_numbers = [340434174857037035, 340434188193324407, 340434188193323500, 340434188193333560, 340434188193343569, 340434154352820677]
for shipment_number in list_of_shipment_numbers:
    main(shipment_number=shipment_number, size=1000, max_workers=4) 

09:12:54 INFO: Generating shipment numbers with seed: 340434174857037035
09:12:54 INFO: Generated 1000 unique shipment numbers
09:12:55 INFO: No events found for shipment number 00340434174857037042
09:12:55 INFO: No events found for shipment number 00340434174857037066
09:12:55 INFO: No events found for shipment number 00340434174857037073
09:12:55 INFO: No events found for shipment number 00340434174857037059
09:12:57 INFO: No events found for shipment number 00340434174857037080
09:12:57 INFO: No events found for shipment number 00340434174857037103
09:12:57 INFO: No events found for shipment number 00340434174857037110
09:12:57 INFO: No events found for shipment number 00340434174857037097
09:12:58 INFO: No events found for shipment number 00340434174857037127
09:12:58 INFO: No events found for shipment number 00340434174857037134
09:12:58 INFO: No events found for shipment number 00340434174857037158
09:12:58 INFO: No events found for shipment number 00340434174857037141
09:12:59 

09:13:31 INFO: No events found for shipment number 00340434174857038186
09:13:31 INFO: No events found for shipment number 00340434174857038179
09:13:32 INFO: No events found for shipment number 00340434174857038193
09:13:32 INFO: No events found for shipment number 00340434174857038209
09:13:33 INFO: No events found for shipment number 00340434174857038216
09:13:33 INFO: No events found for shipment number 00340434174857038223
09:13:33 INFO: No events found for shipment number 00340434174857038230
09:13:34 INFO: No events found for shipment number 00340434174857038247
09:13:34 INFO: No events found for shipment number 00340434174857038254
09:13:34 INFO: No events found for shipment number 00340434174857038261
09:13:34 INFO: No events found for shipment number 00340434174857038278
09:13:35 INFO: No events found for shipment number 00340434174857038285
09:13:35 INFO: No events found for shipment number 00340434174857038292
09:13:35 INFO: No events found for shipment number 0034043417485

09:14:08 INFO: No events found for shipment number 00340434174857039312
09:14:08 INFO: No events found for shipment number 00340434174857039329
09:14:08 INFO: No events found for shipment number 00340434174857039336
09:14:09 INFO: No events found for shipment number 00340434174857039343
09:14:09 INFO: No events found for shipment number 00340434174857039350
09:14:10 INFO: No events found for shipment number 00340434174857039367
09:14:10 INFO: No events found for shipment number 00340434174857039374
09:14:10 INFO: No events found for shipment number 00340434174857039381
09:14:10 INFO: No events found for shipment number 00340434174857039398
09:14:11 INFO: No events found for shipment number 00340434174857039404
09:14:11 INFO: No events found for shipment number 00340434174857039411
09:14:11 INFO: No events found for shipment number 00340434174857039428
09:14:11 INFO: No events found for shipment number 00340434174857039435
09:14:12 INFO: No events found for shipment number 0034043417485

09:14:44 INFO: No events found for shipment number 00340434174857040462
09:14:44 INFO: No events found for shipment number 00340434174857040455
09:14:44 INFO: No events found for shipment number 00340434174857040479
09:14:45 INFO: No events found for shipment number 00340434174857040486
09:14:45 INFO: No events found for shipment number 00340434174857040509
09:14:45 INFO: No events found for shipment number 00340434174857040493
09:14:46 INFO: No events found for shipment number 00340434174857040516
09:14:46 INFO: No events found for shipment number 00340434174857040523
09:14:47 INFO: No events found for shipment number 00340434174857040547
09:14:47 INFO: No events found for shipment number 00340434174857040530
09:14:47 INFO: No events found for shipment number 00340434174857040554
09:14:48 INFO: No events found for shipment number 00340434174857040561
09:14:48 INFO: No events found for shipment number 00340434174857040578
09:14:48 INFO: No events found for shipment number 0034043417485

09:15:20 INFO: No events found for shipment number 00340434174857041599
09:15:21 INFO: No events found for shipment number 00340434174857041605
09:15:21 INFO: No events found for shipment number 00340434174857041629
09:15:21 INFO: No events found for shipment number 00340434174857041612
09:15:22 INFO: No events found for shipment number 00340434174857041636
09:15:22 INFO: No events found for shipment number 00340434174857041643
09:15:23 INFO: No events found for shipment number 00340434174857041650
09:15:23 INFO: No events found for shipment number 00340434174857041667
09:15:23 INFO: No events found for shipment number 00340434174857041674
09:15:24 INFO: No events found for shipment number 00340434174857041681
09:15:24 INFO: No events found for shipment number 00340434174857041698
09:15:24 INFO: No events found for shipment number 00340434174857041704
09:15:24 INFO: No events found for shipment number 00340434174857041711
09:15:25 INFO: No events found for shipment number 0034043417485

...

09:37:43 INFO: Writing file ../data/00340434188193349677.json
09:37:43 INFO: Writing file ../data/00340434188193349684.json
09:37:44 INFO: Writing file ../data/00340434188193349691.json
09:37:44 INFO: Writing file ../data/00340434188193349707.json
09:37:44 INFO: Writing file ../data/00340434188193349714.json
09:37:45 INFO: Writing file ../data/00340434188193349721.json
09:37:45 INFO: Writing file ../data/00340434188193349738.json
09:37:45 INFO: Writing file ../data/00340434188193349745.json
09:37:45 INFO: Writing file ../data/00340434188193349752.json
09:37:46 INFO: Writing file ../data/00340434188193349769.json
09:37:46 INFO: Writing file ../data/00340434188193349776.json
09:37:46 INFO: Writing file ../data/00340434188193349783.json
09:37:47 INFO: Writing file ../data/00340434188193349790.json
09:37:47 INFO: Writing file ../data/00340434188193349806.json
09:37:48 INFO: Writing file ../data/00340434188193349813.json
09:37:48 INFO: Writing file ../data/00340434188193349820.json
09:37:48

09:38:27 INFO: Writing file ../data/00340434188193351007.json
09:38:27 INFO: Writing file ../data/00340434188193351014.json
09:38:27 INFO: Writing file ../data/00340434188193351021.json
09:38:27 INFO: Writing file ../data/00340434188193351038.json
09:38:28 INFO: Writing file ../data/00340434188193351045.json
09:38:28 INFO: Writing file ../data/00340434188193351052.json
09:38:28 INFO: Writing file ../data/00340434188193351069.json
09:38:28 INFO: Writing file ../data/00340434188193351076.json
09:38:29 INFO: Writing file ../data/00340434188193351083.json
09:38:29 INFO: Writing file ../data/00340434188193351090.json
09:38:29 INFO: Writing file ../data/00340434188193351106.json
09:38:30 INFO: Writing file ../data/00340434188193351113.json
09:38:31 INFO: Writing file ../data/00340434188193351120.json
09:38:31 INFO: Writing file ../data/00340434188193351137.json
09:38:31 INFO: Writing file ../data/00340434188193351144.json
09:38:31 INFO: Writing file ../data/00340434188193351151.json
09:38:32

'00340434188193352035' generated an exception: HTTPSConnectionPool(host='www.dhl.de', port=443): Max retries exceeded with url: /int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193352035 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4f5dc595f8>: Failed to establish a new connection: [Errno 111] Connection refused'))
'00340434188193352042' generated an exception: HTTPSConnectionPool(host='www.dhl.de', port=443): Max retries exceeded with url: /int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193352042 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4f5dabcb70>: Failed to establish a new connection: [Errno 111] Connection refused'))
'00340434188193352059' generated an exception: HTTPSConnectionPool(host='www.dhl.de', port=443): Max retries exceeded with url: /int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193352059 (Caused by NewConne

09:40:54 INFO: Writing file ../data/00340434188193352073.json
09:40:54 INFO: Writing file ../data/00340434188193352080.json
09:40:55 INFO: Writing file ../data/00340434188193352097.json
09:40:55 INFO: Writing file ../data/00340434188193352103.json
09:40:55 INFO: Writing file ../data/00340434188193352110.json
09:40:56 INFO: Writing file ../data/00340434188193352127.json
09:40:56 INFO: Writing file ../data/00340434188193352134.json
09:40:56 INFO: Writing file ../data/00340434188193352141.json
09:40:57 INFO: Writing file ../data/00340434188193352158.json
09:40:57 INFO: Writing file ../data/00340434188193352165.json
09:40:57 INFO: Writing file ../data/00340434188193352172.json
09:40:58 INFO: Writing file ../data/00340434188193352189.json
09:40:58 INFO: Writing file ../data/00340434188193352196.json
09:40:58 INFO: Writing file ../data/00340434188193352202.json
09:40:59 INFO: Writing file ../data/00340434188193352219.json
09:40:59 INFO: Writing file ../data/00340434188193352226.json
09:40:59

09:41:37 INFO: Writing file ../data/00340434188193353407.json
09:41:37 INFO: Writing file ../data/00340434188193353414.json
09:41:37 INFO: Writing file ../data/00340434188193353421.json
09:41:38 INFO: Writing file ../data/00340434188193353438.json
09:41:38 INFO: Writing file ../data/00340434188193353445.json
09:41:38 INFO: Writing file ../data/00340434188193353452.json
09:41:38 INFO: Writing file ../data/00340434188193353469.json
09:41:39 INFO: Writing file ../data/00340434188193353476.json
09:41:39 INFO: Writing file ../data/00340434188193353483.json
09:41:40 INFO: Writing file ../data/00340434188193353490.json
09:41:40 INFO: Writing file ../data/00340434188193353506.json
09:41:40 INFO: Writing file ../data/00340434188193353513.json
09:41:40 INFO: Writing file ../data/00340434188193353520.json
09:41:41 INFO: Writing file ../data/00340434188193353537.json
09:41:41 INFO: Writing file ../data/00340434188193353544.json
09:41:41 INFO: Writing file ../data/00340434188193353551.json
09:41:42